In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import sqlalchemy
import psycopg2


ModuleNotFoundError: No module named 'psycopg2'

Train Data

In [2]:
x_train = pd.read_csv('data/train.csv')

In [3]:
x_train.head()

,Year,Season,blueTeamTag,bResult,redTeamTag,gamelength,game_id
0,2016,Summer,TT,1,HWA,50,b96f782c-da04-4641-9306-68968036bfd9
1,2016,Summer,CRW,0,HWA,31,2a7e2cb1-5ac2-4ea2-bd18-0b261e43345a
2,2016,Summer,SUP,1,AUR,50,e04ed84d-929b-4f0e-a769-e7bffc2b5162
3,2016,Spring,JAG,1,SSB,40,8829614a-313e-413c-9520-d95d7b95af31
4,2016,Spring,SIN,0,CHF,36,db2cf98d-a55e-4e28-9702-d7363fad2ebb


Test Data

In [4]:
x_test = pd.read_csv('data/test.csv')

In [5]:
x_test.head()

,Year,Season,blueTeamTag,redTeamTag,gamelength,game_id
0,2017,Spring,IMT,C9,34,9564c32b-dad9-4292-8efb-8fa41f026fe9
1,2017,Summer,SSG,EDG,46,8160b53f-417a-416c-bd66-41b115e91039
2,2017,Summer,JT,ahq,37,5bd6d56b-e337-4f61-83b6-cfe21a481892
3,2017,Spring,PNG,REM,57,614fd189-86c9-4f44-9b13-cf980e1ccc9f
4,2017,Spring,C9,DIG,46,539f6975-88f2-40ea-8d22-4b500370e971


In [6]:
# Db settings - PostgreSQL
username = 'ldsa_student'
password = 'R4Fr4P3aAgMYBqqP'  
host_name = 'batch4-s02-db-instance.ctq2kxc7kx1i.eu-west-1.rds.amazonaws.com'
port = 5432
db_name = 'batch4_s02_db'
schema = 'hackathon_students'

conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)
conn_args = {'options': '-csearch_path={}'.format(schema)}

In [7]:
engine = sqlalchemy.create_engine(conn_str, connect_args=conn_args)

Monster columns raw

In [8]:
query = 'select "Team" as monster_name,"Time" as time_to_dist_monster,"game_id" from monsters;'
monsters_columns_= pd.read_sql_query(query, engine)
monsters_columns_.head()

,monster_name,time_to_dist_monster,game_id
0,bDragons,4.656,258c77d0-6226-4243-892a-8d9de5a5b401
1,bDragons,10.084,3c1cc1d2-0ff4-4559-95e2-99cba51eadce
2,bDragons,7.220,62d0c6cd-77a0-4fb4-a70b-3adad90d0247
3,bDragons,10.998,49e39636-5df8-4b54-96fb-133f3f336e55
4,bDragons,10.085,54fbed5e-33f8-4f16-92d7-7f00e8dda7ba


In [9]:
monsters_columns_.to_csv('monsters_columns_.csv')

Tower columns raw

In [10]:
query2 = 'select "Team" as towers_name,"Time" as time_to_dist_tower ,"game_id" from towers order by "index";'
towers_columns_= pd.read_sql_query(query2, engine)
towers_columns_.head()

,towers_name,time_to_dist_tower,game_id
0,bTowers,6.378,5f86d932-465d-4278-8f2c-e837398c696e
1,bTowers,8.946,4b4c0c9f-62e2-44ea-a2fe-aa4c59a182b6
2,bTowers,8.960,3beaa8b3-6263-47c9-85fa-8dde611f0bb6
3,bTowers,10.332,9dcadc3c-7c30-4773-af55-a73edb4cf589
4,bTowers,8.743,3ca49512-a10f-472a-8c15-3bb67d6dbed3


In [11]:
towers_columns_.to_csv('towers_columns_.csv')

Total monster kill per team

In [12]:
#Total monsters per team
#B team
total_monsters_b_team = monsters_columns_[monsters_columns_['monster_name'].isin({'bHeralds','bDragons'})].groupby('game_id').size()
total_monsters_b_team = pd.DataFrame(total_monsters_b_team).reset_index()
total_monsters_b_team.columns = ['game_id','total_monsters_b']
#R team
total_monsters_r_team = monsters_columns_[monsters_columns_['monster_name'].isin({'rHeralds','rDragons'})].groupby('game_id').size()
total_monsters_r_team = pd.DataFrame(total_monsters_r_team).reset_index()
total_monsters_r_team.columns = ['game_id','total_monsters_a']
monsters_columns_grouped = pd.merge(total_monsters_b_team, total_monsters_r_team, on = 'game_id', how ='outer').fillna(0)
 


In [13]:
monsters_columns_grouped

,game_id,total_monsters_b,total_monsters_a
0,0024055a-f824-4a82-818f-22443bd2e080,1.0,0.0
1,0043fd49-2f53-46e1-8bce-f4da7a46212b,1.0,0.0
2,004a6723-f15c-439f-9615-0babb56a62cd,1.0,0.0
3,006682af-c158-40eb-9073-134b68c8d696,1.0,1.0
4,0073723e-a0f3-41a6-8fec-d55a9d02e207,1.0,0.0
...,...,...,...
3323,ff7aa9cb-ec91-4571-bafb-b6e4db32c15c,0.0,1.0
3324,ffb10df5-de4b-4db6-bb27-3e805ddd17a7,0.0,1.0
3325,ffbf8ba5-f8b9-47ec-87aa-0776c1cc053e,0.0,1.0
3326,ffc83dc9-b51c-4698-9963-650653ddd8e0,0.0,1.0


In [14]:
monsters_columns_grouped.to_csv('monsters_columns_grouped.csv', index=False) 

total towers distroy by team

In [15]:
#Total towers per team
#B team
total_towers_b_team = towers_columns_[towers_columns_['towers_name'] == 'bTowers'].groupby('game_id').size()
total_towers_b_team = pd.DataFrame(total_towers_b_team).reset_index()
total_towers_b_team.columns = ['game_id','total_towers_b_team']
#R team
total_towers_r_team = towers_columns_[towers_columns_['towers_name'] == 'rTowers'].groupby('game_id').size()
total_towers_r_team = pd.DataFrame(total_towers_r_team).reset_index()
total_towers_r_team.columns = ['game_id','total_towers_r_team']
#R team
towers_columns_grouped = pd.merge(total_towers_b_team, total_towers_r_team, on = 'game_id', how ='outer').fillna(0)



In [16]:
towers_columns_grouped 

,game_id,total_towers_b_team,total_towers_r_team
0,00032e36-8a18-472c-bf38-61b836b72e45,2.0,1.0
1,00265d98-0c54-4c63-935e-7367e934f06f,1.0,0.0
2,00281254-1bf8-42c0-9af5-9e54c6b36616,1.0,0.0
3,006682af-c158-40eb-9073-134b68c8d696,2.0,3.0
4,0073723e-a0f3-41a6-8fec-d55a9d02e207,1.0,1.0
...,...,...,...
3038,fec1a782-04fe-408c-83d8-ca3b419ced43,0.0,1.0
3039,fed8c9b7-521e-426d-93cc-35c0470f763e,0.0,1.0
3040,ff22d1fd-ecf7-4ec6-9325-1e95027ddaec,0.0,1.0
3041,ff7dcd1c-4f43-4ea2-ae4d-20702cfa8ae3,0.0,1.0


In [17]:
towers_columns_grouped.to_csv('towers_columns_grouped.csv', index=False)  

In [20]:
! explorer.exe .